In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Load the training dataset
df_train = pd.read_csv('D:\FPTUni\SP24\ADY201m\Lab05\ADY201m_Lab05_SE183256/train.csv')
df_test = pd.read_csv('D:\FPTUni\SP24\ADY201m\Lab05\ADY201m_Lab05_SE183256/test.csv')
# Display a few rows of the training dataset
df_train.head()

# Separate features and target variable
X = df_train.drop("SalePrice", axis=1)
y = df_train["SalePrice"]

# Define numerical and categorical features
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# Define preprocessing steps for numerical and categorical features
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Define the model
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Bundle preprocessing and modeling code in a pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                            ('model', model)])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the model to the training data
pipeline.fit(X_train, y_train)

# Make predictions on the test set
predictions = pipeline.predict(X_test)

# Evaluate the model performance using Mean Squared Error
mse = mean_squared_error(y_test, predictions)

# Print the Root Mean Squared Error
print("Root Mean Squared Error:", np.sqrt(mse))

test_predictions = pipeline.predict(df_test)

# Create a submission DataFrame
submission_df = pd.DataFrame({'Id': df_test['Id'], 'SalePrice': test_predictions})

# Save the submission DataFrame to a CSV file
submission_df.to_csv('submission_rf_preprocessed.csv', index=False)

Root Mean Squared Error: 29285.518968142213
